In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

tw = pd.read_csv("./tw.csv") # read tw.csv
# us = pd.read_csv("./us.csv")
tw.head(3)
# tw.columns

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,income,tag_count,desc_length,trending_days,duration
0,--d-f3lIS6g,吃完直接重X公斤？挑戰一天吃20000大卡！爆食日放開吃｜結果一發不可收拾？胖豪豪｜超罪惡｜...,2020-12-24 09:30:01,UCa2YiSXNTkmOA-QTKdzzbSQ,吃貨豪豪HowHowEat,22,2020-12-25,大胃王|大胃王挑戰|大食<U+3044>|Food Challenge|Challenge|...,136300,3278,...,428,https://i.ytimg.com/vi/--d-f3lIS6g/default.jpg,False,False,喜歡看我吃東西記得按讚&訂閱喔下次想看我吃什麼可以底下留言告訴我IG有我的一些生活有興趣的朋...,80.41700,74,223,1,1394
1,-4ni0pcKZ4w,I CAN'T STOP ME (English Ver.),2020-11-30 04:49:15,UCAq0pFGa2w9SjxOq0ZxKVIw,TWICE - Topic,10,2020-12-06,TWICE|I CAN'T STOP ME (English Ver.),2104241,276409,...,19064,https://i.ytimg.com/vi/-4ni0pcKZ4w/default.jpg,False,False,Provided to YouTube by JYP Entertainment Corpo...,1241.50219,2,523,6,206
2,-5q5mZbe3V8,BTS (<U+BC29><U+D0C4><U+C18C><U+B144><U+B2E8>)...,2020-11-20 04:58:11,UC3IZKseVpdzPSBaWxBxundA,Big Hit Labels,10,2020-12-07,BIGHIT|<U+BE45><U+D788><U+D2B8>|<U+BC29><U+D0C...,177827170,11946371,...,4301497,https://i.ytimg.com/vi/-5q5mZbe3V8/default.jpg,False,False,BTS (<U+BC29><U+D0C4><U+C18C><U+B144><U+B2E8>)...,104918.03030,6,1384,17,231


In [12]:
y1 = tw["view_count"] # 預測目標 1：view_count
ids = tw["video_id"] # video id
# 移除不需要的欄位
tw.drop(["video_id", "title", "publishedAt", "channelId", "channelTitle", "trending_date", "tags", "thumbnail_link", "comments_disabled", "ratings_disabled", "description"], axis = 1, inplace = True)

In [13]:
tw.head()

,categoryId,view_count,likes,dislikes,comment_count,income,tag_count,desc_length,trending_days,duration
0,22,136300,3278,101,428,80.41700,74,223,1,1394
1,10,2104241,276409,2851,19064,1241.50219,2,523,6,206
2,10,177827170,11946371,154679,4301497,104918.03030,6,1384,17,231
3,10,1384571,41631,529,2810,816.89689,37,1519,6,226
4,24,2435428,107030,1624,5688,1436.90252,59,272,4,269


In [14]:
# 相關係數圖表，觀察目標值與其他特徵之相關係數
corr = tw.corr()
corr.style.background_gradient(cmap = "coolwarm").set_precision(2)

,categoryId,view_count,likes,dislikes,comment_count,income,tag_count,desc_length,trending_days,duration
categoryId,1.00,-0.14,-0.17,-0.13,-0.09,-0.14,0.14,-0.12,-0.18,0.06
view_count,-0.14,1.00,0.86,0.89,0.81,1.00,-0.03,0.15,0.18,0.09
likes,-0.17,0.86,1.00,0.83,0.86,0.86,-0.04,0.12,0.15,-0.02
dislikes,-0.13,0.89,0.83,1.00,0.72,0.89,-0.03,0.12,0.17,0.03
comment_count,-0.09,0.81,0.86,0.72,1.00,0.81,-0.03,0.06,0.12,-0.01
income,-0.14,1.00,0.86,0.89,0.81,1.00,-0.03,0.15,0.18,0.09
tag_count,0.14,-0.03,-0.04,-0.03,-0.03,-0.03,1.00,0.11,-0.09,-0.06
desc_length,-0.12,0.15,0.12,0.12,0.06,0.15,0.11,1.00,0.14,-0.06
trending_days,-0.18,0.18,0.15,0.17,0.12,0.18,-0.09,0.14,1.00,-0.09
duration,0.06,0.09,-0.02,0.03,-0.01,0.09,-0.06,-0.06,-0.09,1.00


In [15]:
# 篩選相關係數大於 0.8 的特徵 (高度正相關)
high_corr = list(corr[(corr["view_count"] > 0.8)].index)
high_corr.pop(0)
high_corr.pop(3)
high_corr

['likes', 'dislikes', 'comment_count']

In [16]:
tw.drop(["view_count", "income"], axis = 1, inplace = True)
tw.head()

,categoryId,likes,dislikes,comment_count,tag_count,desc_length,trending_days,duration
0,22,3278,101,428,74,223,1,1394
1,10,276409,2851,19064,2,523,6,206
2,10,11946371,154679,4301497,6,1384,17,231
3,10,41631,529,2810,37,1519,6,226
4,24,107030,1624,5688,59,272,4,269


In [17]:
# min-max scaling
for c in tw.columns:
    tw[c] = MinMaxScaler().fit_transform(tw[c].values.reshape(-1, 1))
tw.head()

,categoryId,likes,dislikes,comment_count,tag_count,desc_length,trending_days,duration
0,0.777778,0.000274,0.000653,0.000100,0.524823,0.047650,0.0000,0.003157
1,0.333333,0.023137,0.018432,0.004432,0.014184,0.111752,0.3125,0.000443
2,0.333333,1.000000,1.000000,1.000000,0.042553,0.295726,1.0000,0.000500
3,0.333333,0.003485,0.003420,0.000653,0.262411,0.324573,0.3125,0.000489
4,0.851852,0.008959,0.010499,0.001322,0.418440,0.058120,0.1875,0.000587


In [18]:
# 輸出 scaling 後之資料集 : new_tw.csv
tw.insert(0, "video_id", ids)
tw["view_count"] = y1
tw.to_csv("new_tw.csv", index = False)

In [19]:
# scaling 後之資料集
new_tw = pd.read_csv("new_tw.csv")
new_tw.head()

,video_id,categoryId,likes,dislikes,comment_count,tag_count,desc_length,trending_days,duration,view_count
0,--d-f3lIS6g,0.777778,0.000274,0.000653,0.000100,0.524823,0.047650,0.0000,0.003157,136300
1,-4ni0pcKZ4w,0.333333,0.023137,0.018432,0.004432,0.014184,0.111752,0.3125,0.000443,2104241
2,-5q5mZbe3V8,0.333333,1.000000,1.000000,1.000000,0.042553,0.295726,1.0000,0.000500,177827170
3,-607OwI4gPo,0.333333,0.003485,0.003420,0.000653,0.262411,0.324573,0.3125,0.000489,1384571
4,-6zExiuIotU,0.851852,0.008959,0.010499,0.001322,0.418440,0.058120,0.1875,0.000587,2435428
